In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Fnac\Desktop\Trello ML\teaching_ml_2023\en.openfoodfacts.org.products.csv", nrows = 10000, sep='\t', encoding='utf-8',low_memory=False)
df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0011150039840,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489075957,2017-03-09T16:12:37Z,1489075958,2017-03-09T16:12:38Z,Great Northern Beans,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,0011150039857,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489053597,2017-03-09T09:59:57Z,1489053598,2017-03-09T09:59:58Z,Drink Mix Packets,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,0011150039864,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489053269,2017-03-09T09:54:29Z,1587572617,2020-04-22T16:23:37Z,Green Split Peas,NaN,NaN,...,NaN,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,0011150039871,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489053597,2017-03-09T09:59:57Z,1489053597,2017-03-09T09:59:57Z,Drink Mix Packets,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def select_columns(df, num_cols=None, ordinal_cols=None, nominal_cols=None, cat_threshold=50):
    """
    --------------------------------------------------------------------------
    Goals : Filter and select columns based on their data types and categories
    --------------------------------------------------------------------------
    Parameters
    --------------------------------------------------------------------------
        +df: DataFrame to select columns from
        +num_cols: List of column names for numeric variables
        +ordinal_cols: List of column names for ordinal categorical variables
        +nominal_cols: List of column names for nominal categorical variables
        +cat_threshold: Maximum number of categories for nominal variables
    -------------------------------------------------------------------------
    En sortie: DataFrame with selected columns
    -------------------------------------------------------------------------
    @Author: GBE Grâce
    """
    
    # Convert numeric columns with low cardinality to int or float for memory optimization
    if num_cols:
        for col in num_cols:
            if np.issubdtype(df[col].dtype, np.integer):
                if df[col].nunique() < len(df) * 0.5:
                    df[col] = pd.to_numeric(df[col], downcast='integer')
                else:
                    df[col] = pd.to_numeric(df[col], downcast='signed')
            else:
                if df[col].nunique() < len(df) * 0.5:
                    df[col] = pd.to_numeric(df[col], downcast='float')
                else:
                    df[col] = pd.to_numeric(df[col], downcast='signed')

    # Select numeric columns
    if num_cols:
        num_df = df[num_cols]
    else:
        num_df = pd.DataFrame()

    # Select ordinal categorical columns
    if ordinal_cols:
        ordinal_df = df[ordinal_cols].astype('category')
    else:
        ordinal_df = pd.DataFrame()

    # Select nominal categorical columns with low cardinality
    if nominal_cols:
        nominal_df = pd.DataFrame()
        for col in nominal_cols:
            if df[col].nunique() <= cat_threshold:
                nominal_df[col] = df[col].astype('category')
    else:
        nominal_df = pd.DataFrame()

    # Combine selected columns into one DataFrame
    selected_df = pd.concat([num_df, ordinal_df, nominal_df], axis=1)

    return selected_df


num_cols = ['energy_100g', 'fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g']
ordinal_cols = ['nutrition-score-fr_100g']
nominal_cols = ['brands', 'countries', 'categories', 'labels']

selected_df = select_columns(df, num_cols=num_cols, ordinal_cols=ordinal_cols, nominal_cols=nominal_cols, cat_threshold=50)

selected_df